In [178]:
import numpy as np
from astropy.io import fits
from astropy.table import Table
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [273]:
from commands import ingest_files, calibration_command, reduction_command

To "de"-certify the calibration files:

In [274]:
import lsst.daf.butler
from lsst.daf.butler import Butler
repo_path = '/scratch/gpfs/JENNYG/jiaxuanl/HSC/repo'

# writeable_butler = Butler(
#     repo_path, writeable=True
# )

# writeable_butler.registry.decertify(
#     collection='HSC/calib/dw1008p7058',
#     datasetType='sky',
#     timespan=lsst.daf.butler.Timespan(None, None),
# )

# dw1336p5226

In [275]:
# Example usage
expid_dict = {'flat': [(196834, 196853), (203400, 203419)], 
              'science': [(197336, 197341), (197342, 197343), (206932, 206945)],
              'bias': [(196854, 196873), (202548, 202567)],
              'dark': [(196874, 196883), (202568, 202577)]}

print(ingest_files("dw1336p5226", expid_dict['bias'], filetype='bias'))
print(ingest_files("dw1336p5226", expid_dict['flat'], filetype='flat'))
print(ingest_files("dw1336p5226", expid_dict['dark'], filetype='dark'))
print(ingest_files("dw1336p5226", expid_dict['science'], filetype='science'))

######## Ingesting bias... ##########
mkdir -p /scratch/gpfs/JENNYG/jiaxuanl/HSC/raw_smoka/dw1336p5226/bias
for expid in {196854..196873} {202548..202567}; do
    mv /scratch/gpfs/JENNYG/jiaxuanl/HSC/raw_smoka/dw1336p5226/HSCA${expid}*.fits \
       /scratch/gpfs/JENNYG/jiaxuanl/HSC/raw_smoka/dw1336p5226/bias/
done

LOGFILE=$LOGDIR/dw1336p5226_ingest_bias.log; \
FILES=/scratch/gpfs/JENNYG/jiaxuanl/HSC/raw_smoka/dw1336p5226/bias/*.fits; \
date | tee $LOGFILE; \
butler ingest-raws $REPO $FILES --transfer link \
2>&1 | tee -a $LOGFILE; \
date | tee -a $LOGFILE

######## Ingesting flat... ##########
mkdir -p /scratch/gpfs/JENNYG/jiaxuanl/HSC/raw_smoka/dw1336p5226/flat
for expid in {196834..196853} {203400..203419}; do
    mv /scratch/gpfs/JENNYG/jiaxuanl/HSC/raw_smoka/dw1336p5226/HSCA${expid}*.fits \
       /scratch/gpfs/JENNYG/jiaxuanl/HSC/raw_smoka/dw1336p5226/flat/
done

LOGFILE=$LOGDIR/dw1336p5226_ingest_flat.log; \
FILES=/scratch/gpfs/JENNYG/jiaxuanl/HSC/raw_smoka/dw1336p5226/flat/*.f

### Calibrations

In [276]:
galaxy_name = "dw1336p5226"

command = calibration_command(galaxy_name, expid_dict, filetype="sky", clobber=True)
print(command)

######## Generating sky calibration command... ##########
export SKYEXPS="(197336, 197338, 197340, 197342, 206932, 206934, 206936, 206938, 206940, 206942, 206944)"
LOGFILE=$LOGDIR/dw1336p5226_sky.log; \
date | tee $LOGFILE; \
pipetask --long-log run --register-dataset-types -j 36 \
-b $REPO --instrument lsst.obs.subaru.HyperSuprimeCam \
-i HSC/raw/all,HSC/calib,HSC/calib/unbounded,HSC/calib/dw1336p5226,HSC/calib/curated/19700101T000000Z \
-o HSC/calib/dw1336p5226/sky \
-p $CP_PIPE_DIR/pipelines/HSC/cpSky.yaml \
-d "instrument='HSC' AND exposure IN $SKYEXPS AND detector!=9" \
--clobber-outputs --extend-run --skip-existing \
2>&1 | tee -a $LOGFILE; \
date | tee -a $LOGFILE\

butler certify-calibrations $REPO HSC/calib/dw1336p5226/sky HSC/calib/dw1336p5226 sky --begin-date 2016-01-01T00:00:00 --end-date 2024-06-30T23:59:59



### Reduction

In [278]:
print('salloc --nodes=1 --ntasks=1 --mem=32G --time=08:00:00 --account=merian \n')

print(reduction_command('dw1336p5226', expid_dict, 'step1'))

tracts = (16817, 16818, 16968, 16969)
print(reduction_command('dw1336p5226', expid_dict, 'step3a', tracts=tracts, clobber=True))

salloc --nodes=1 --ntasks=1 --mem=32G --time=08:00:00 --account=merian 

######## ...step1 command... #########
SCIEXPS="(197336, 197338, 197340, 197342, 206932, 206934, 206936, 206938, 206940, 206942, 206944)"; \
LOGFILE=$LOGDIR/dw1336p5226_step1.log; \
BADCCDS="(9, 33)"; \
date | tee $LOGFILE; \
pipetask --long-log run --register-dataset-types -j 36 \
-b $REPO --instrument lsst.obs.subaru.HyperSuprimeCam \
-i HSC/raw/all,HSC/calib/dw1336p5226,HSC/calib,HSC/calib/unbounded,refcats/gen2,skymaps \
-o HSC/runs/dw1336p5226 \
-p $DRP_PIPE_DIR/pipelines/HSC/DRP-RC2.yaml#step1 \
-d "instrument='HSC' AND exposure IN $SCIEXPS AND detector NOT IN $BADCCDS" \
2>&1 | tee -a $LOGFILE; \
date | tee -a $LOGFILE\

######## ...step3a command... #########
SCIEXPS="(197336, 197338, 197340, 197342, 206932, 206934, 206936, 206938, 206940, 206942, 206944)"; \
LOGFILE=$LOGDIR/dw1336p5226_step3a.log; \
BADCCDS="(9, 33)"; \
TRACTS="(16817, 16818, 16968, 16969)"; \
date | tee $LOGFILE; \
pipetask --long-log ru

## Get a computing node:


## Step 1
```bash
SCIEXPS="(197336, 197338, 197340, 197342, 206932, 206934, 206936, 206938, 206940, 206942, 206944)" \
OUTPUT=HSC/runs/dw1336p5226 \
LOGFILE=$LOGDIR/dw1336p5226_step1.log; \
date | tee $LOGFILE; \
pipetask --long-log run --register-dataset-types -j 48 \
-b $REPO --instrument lsst.obs.subaru.HyperSuprimeCam \
-i HSC/raw/all,HSC/calib/dw1336p5226,HSC/calib,HSC/calib/unbounded,refcats/gen2,skymaps \
-o $OUTPUT \
-p $DRP_PIPE_DIR/pipelines/HSC/DRP-RC2.yaml#step1 \
-d "instrument='HSC' AND exposure in $SCIEXPS AND detector!=9 AND detector!=33" \
--clobber-outputs --extend-run --skip-existing \
2>&1 | tee -a $LOGFILE; \
date | tee -a $LOGFILE
```

## Step 2a
```bash
BADCCDS="(9, 33, 104, 105, 106, 107, 108, 109, 110, 111)" \
SCIEXPS="(197336, 197338, 197340, 197342, 206932, 206934, 206936, 206938, 206940, 206942, 206944)" \
OUTPUT=HSC/runs/dw1336p5226 \
LOGFILE=$LOGDIR/dw1336p5226_step2a.log; \
date | tee $LOGFILE; \
pipetask --long-log run --register-dataset-types -j 48 \
-b $REPO --instrument lsst.obs.subaru.HyperSuprimeCam \
-i HSC/raw/all,HSC/calib/dw1336p5226,HSC/calib,HSC/calib/unbounded,refcats/gen2,skymaps \
-o $OUTPUT \
-p $DRP_PIPE_DIR/pipelines/HSC/DRP-RC2.yaml#step2a \
-d "instrument='HSC' AND exposure IN $SCIEXPS AND detector NOT IN $BADCCDS" \
2>&1 | tee -a $LOGFILE; \
date | tee -a $LOGFILE
```

## Step 2b

```
pipetask build -p $REPO/pipelines/step2b.yaml#step2b --show config | less
```

```bash
OUTPUT=HSC/runs/dw1336p5226 \
LOGFILE=$LOGDIR/dw1336p5226_step2b.log; \
date | tee $LOGFILE; \
pipetask --long-log run --register-dataset-types -j 48 \
-b $REPO --instrument lsst.obs.subaru.HyperSuprimeCam \
-i HSC/raw/all,HSC/calib/dw1336p5226,HSC/calib,HSC/calib/unbounded,refcats/gen2,skymaps \
-o $OUTPUT \
-p $REPO/pipelines/step2b.yaml#step2b \
-d "instrument='HSC' AND skymap='hsc_rings_v1'" \
-c jointcal:connections.astrometryRefCat='gaia_dr3_20230707' \
-c jointcal:maxAstrometrySteps=50 \
-c jointcal:doPhotometry=True \
2>&1 | tee -a $LOGFILE; \
date | tee -a $LOGFILE
```

**Step 2c:**

```bash
OUTPUT=HSC/runs/dw1336p5226 \
LOGFILE=$LOGDIR/dw1336p5226_step2c.log; \
date | tee $LOGFILE; \
pipetask --long-log run --register-dataset-types -j 24 \
-b $REPO --instrument lsst.obs.subaru.HyperSuprimeCam \
-i HSC/raw/all,HSC/calib/dw1336p5226,HSC/calib,HSC/calib/unbounded,refcats/gen2,skymaps \
-o $OUTPUT \
-p $REPO/pipelines/step2ce.yaml#step2c \
-c updateVisitSummary:wcs_provider="input_summary" \
-c updateVisitSummary:photo_calib_provider="input_summary" \
-c updateVisitSummary:background_provider="input_summary" \
-d "instrument='HSC' AND skymap='hsc_rings_v1' AND visit IN $SCIEXPS" \
2>&1 | tee -a $LOGFILE; \
date | tee -a $LOGFILE
```

**Step 2e:**
```bash
OUTPUT=HSC/runs/dw1336p5226 \
LOGFILE=$LOGDIR/dw1336p5226_step2e.log; \
date | tee $LOGFILE; \
pipetask --long-log run --register-dataset-types -j 24 \
-b $REPO --instrument lsst.obs.subaru.HyperSuprimeCam \
-i HSC/raw/all,HSC/calib/dw1336p5226,HSC/calib,HSC/calib/unbounded,refcats/gen2,skymaps \
-o $OUTPUT \
-p $REPO/pipelines/step2ce.yaml#step2e \
-d "instrument='HSC' AND skymap='hsc_rings_v1'" \
2>&1 | tee -a $LOGFILE; \
date | tee -a $LOGFILE
```


**Step 3a:**

```bash
pipetask build -p $REPO/pipelines/step3a.yaml --show config | less

pipetask build \
-p $REPO/pipelines/step3a.yaml#step3a \
--show pipeline \
--pipeline-dot ./pipeline.dot; \
dot ./pipeline.dot -Tpdf > $LOGDIR/pipeline_step3a.pdf
cp $LOGDIR/pipeline_step3a.pdf /tigress/jiaxuanl/public_html/LSST/HSC/NGC4605/

```
<!-- , 17115, 17114 -->

```bash
TRACTS="(16817, 16818, 16968, 16969)" \
OUTPUT=HSC/runs/dw1336p5226 \
LOGFILE=$LOGDIR/dw1336p5226_step3a.log; \
date | tee $LOGFILE; \
pipetask --long-log run --register-dataset-types -j 24 \
-b $REPO --instrument lsst.obs.subaru.HyperSuprimeCam \
-i HSC/raw/all,HSC/calib/dw1336p5226,HSC/calib,HSC/calib/unbounded,refcats/gen2,skymaps \
-o $OUTPUT \
-p $REPO/pipelines/step3a.yaml#step3a \
-d "instrument='HSC' AND skymap='hsc_rings_v1' AND tract IN $TRACTS" \
--clobber-outputs --extend-run --skip-existing \
2>&1 | tee -a $LOGFILE; \
date | tee -a $LOGFILE
```

# dw1008p7058

dw1008p7058: `152.14053932599145, 70.97524696548678`. The overlapping files are (g and z-band): [‘HSCA28522938.fits’, ‘HSCA28465317.fits’, ‘HSCA27522030.fits’, ‘HSCA28523218.fits’, ‘HSCA20333021.fits’]. Also this is very likely cirrus.
Lets download **only z-band** because cirrus is dark. 


 - science frames: 284652, 285228, 285232, 308696
 - domeflats: 284018..284037, 306756..306775
 <!-- - bias: 281054..281075 -->
 - bias: 284004, 06, 08, 10, 12, 284554, 56, 58, 60, 62, 284566, 68, 70, 72, 74, 285160, 62, 64, 66, 68, 285148, 50, 52, 54, 56, 285736, 38, 40, 42, 44
 - dark: 284016, 284564, 284578, 285158, 285172, 285746
 <!-- - dark: 281076..281311 -->
- `P03jiaxuanl1228052549NT`

    HSCA284652XX 2022-06-06
    HSCA285228XX 2022-06-07
    HSCA285232XX 2022-06-07
    HSCA308696XX 2023-05-24

In [164]:
# dw1008p7058
expid_dict = {'bias': [(284004, 284012+1), (284554, 284562+1), (284566, 284574+1), (285160, 285168+1), (285148, 285156+1), (285736, 285744+1)],
              'flat': [(284018, 284037), (306756, 306775)], 
              'dark': [(284016, 284016+1), (284564, 284564+1), (284578, 284578+1), (285158, 285158+1), (285172, 285172+1), (285746, 285746+1)],
              'science': [(284652, 284653), (285228, 285229), (285232, 285233), (308696, 308697)]}

print(ingest_files("dw1008p7058", expid_dict['bias'], filetype='bias'))
print(ingest_files("dw1008p7058", expid_dict['flat'], filetype='flat'))
print(ingest_files("dw1008p7058", expid_dict['dark'], filetype='dark'))
print(ingest_files("dw1008p7058", expid_dict['science'], filetype='science'))

######## Ingesting bias... ##########
mkdir -p /scratch/gpfs/JENNYG/jiaxuanl/HSC/raw_smoka/dw1008p7058/bias
for expid in {284004..284013} {284554..284563} {284566..284575} {285160..285169} {285148..285157} {285736..285745}; do
    mv /scratch/gpfs/JENNYG/jiaxuanl/HSC/raw_smoka/dw1008p7058/HSCA${expid}*.fits \
       /scratch/gpfs/JENNYG/jiaxuanl/HSC/raw_smoka/dw1008p7058/bias/
done

LOGFILE=$LOGDIR/dw1008p7058_ingest_bias.log; \
FILES=/scratch/gpfs/JENNYG/jiaxuanl/HSC/raw_smoka/dw1008p7058/bias/*.fits; \
date | tee $LOGFILE; \
butler ingest-raws $REPO $FILES --transfer link \
2>&1 | tee -a $LOGFILE; \
date | tee -a $LOGFILE

######## Ingesting flat... ##########
mkdir -p /scratch/gpfs/JENNYG/jiaxuanl/HSC/raw_smoka/dw1008p7058/flat
for expid in {284018..284037} {306756..306775}; do
    mv /scratch/gpfs/JENNYG/jiaxuanl/HSC/raw_smoka/dw1008p7058/HSCA${expid}*.fits \
       /scratch/gpfs/JENNYG/jiaxuanl/HSC/raw_smoka/dw1008p7058/flat/
done

LOGFILE=$LOGDIR/dw1008p7058_ingest_flat.log; \
FI

In [176]:
galaxy_name = "dw1008p7058"

command = calibration_command(galaxy_name, expid_dict, filetype="sky", clobber=True)
print(command)

######## Generating sky calibration command... ##########
export SKYEXPS="(284652, 285228, 285232, 308696)"
LOGFILE=$LOGDIR/dw1008p7058_sky.log; \
date | tee $LOGFILE; \
pipetask --long-log run --register-dataset-types -j 36 \
-b $REPO --instrument lsst.obs.subaru.HyperSuprimeCam \
-i HSC/raw/all,HSC/calib,HSC/calib/unbounded,HSC/calib/dw1008p7058,HSC/calib/curated/19700101T000000Z \
-o HSC/calib/dw1008p7058/sky \
-p $CP_PIPE_DIR/pipelines/HSC/cpSky.yaml \
-d "instrument='HSC' AND exposure IN $SKYEXPS AND detector!=9" \
--clobber-outputs --extend-run --skip-existing \
2>&1 | tee -a $LOGFILE; \
date | tee -a $LOGFILE\

butler certify-calibrations $REPO HSC/calib/dw1008p7058/sky HSC/calib/dw1008p7058 sky --begin-date 2016-01-01T00:00:00 --end-date 2024-06-30T23:59:59



    SKYEXPS="(284652, 285228, 285232, 308696)" \
    BADCCDS="(9, 33, 104, 105, 106, 107, 108, 109, 110, 111)" \
    LOGFILE=$LOGDIR/dw1008p7058_sky.log; \
    date | tee $LOGFILE; \
    pipetask --long-log run --register-dataset-types -j 36 \
    -b $REPO --instrument lsst.obs.subaru.HyperSuprimeCam \
    -i HSC/raw/all,HSC/calib,HSC/calib/unbounded,HSC/calib/dw1008p7058,HSC/calib/curated/19700101T000000Z \
    -o HSC/calib/dw1008p7058/sky \
    -p $CP_PIPE_DIR/pipelines/HSC/cpSky.yaml \
    -d "instrument='HSC' AND exposure IN $SKYEXPS AND detector NOT IN $BADCCDS" \
    --clobber-outputs --extend-run --skip-existing \
    2>&1 | tee -a $LOGFILE; \
    date | tee -a $LOGFILE\

**Get a computing node:**
`salloc --nodes=1 --ntasks=1 --mem=32G --time=08:00:00 --account=merian`


**Step 1**

```bash
SCIEXPS="(284652, 285228, 285232, 308696)" \
OUTPUT=HSC/runs/dw1008p7058 \
LOGFILE=$LOGDIR/dw1008p7058_step1.log; \
date | tee $LOGFILE; \
pipetask --long-log run --register-dataset-types -j 48 \
-b $REPO --instrument lsst.obs.subaru.HyperSuprimeCam \
-i HSC/raw/all,HSC/calib/dw1008p7058,HSC/calib,HSC/calib/unbounded,refcats/gen2,skymaps \
-o $OUTPUT \
-p $DRP_PIPE_DIR/pipelines/HSC/DRP-RC2.yaml#step1 \
-d "instrument='HSC' AND exposure IN $SCIEXPS AND detector!=9" \
--clobber-outputs --extend-run --skip-existing \
2>&1 | tee -a $LOGFILE; \
date | tee -a $LOGFILE
```

**Step 2a**
```bash
BADCCDS="(9, 33, 104, 105, 106, 107, 108, 109, 110, 111)" \
SCIEXPS="(284652, 285228, 285232, 308696)" \
OUTPUT=HSC/runs/dw1008p7058 \
LOGFILE=$LOGDIR/dw1008p7058_step2a.log; \
date | tee $LOGFILE; \
pipetask --long-log run --register-dataset-types -j 48 \
-b $REPO --instrument lsst.obs.subaru.HyperSuprimeCam \
-i HSC/raw/all,HSC/calib/dw1008p7058,HSC/calib,HSC/calib/unbounded,refcats/gen2,skymaps \
-o $OUTPUT \
-p $DRP_PIPE_DIR/pipelines/HSC/DRP-RC2.yaml#step2a \
-d "instrument='HSC' AND exposure IN $SCIEXPS AND detector NOT IN $BADCCDS" \
2>&1 | tee -a $LOGFILE; \
date | tee -a $LOGFILE
```

**Step 2b**

```
pipetask build -p $REPO/pipelines/step2b.yaml#step2b --show config | less
```

```bash
OUTPUT=HSC/runs/dw1008p7058 \
LOGFILE=$LOGDIR/dw1008p7058_step2b.log; \
date | tee $LOGFILE; \
pipetask --long-log run --register-dataset-types -j 48 \
-b $REPO --instrument lsst.obs.subaru.HyperSuprimeCam \
-i HSC/raw/all,HSC/calib/dw1008p7058,HSC/calib,HSC/calib/unbounded,refcats/gen2,skymaps \
-o $OUTPUT \
-p $REPO/pipelines/step2b.yaml#step2b \
-d "instrument='HSC' AND skymap='hsc_rings_v1'" \
-c jointcal:connections.astrometryRefCat='gaia_dr3_20230707' \
-c jointcal:maxAstrometrySteps=50 \
-c jointcal:doPhotometry=True \
2>&1 | tee -a $LOGFILE; \
date | tee -a $LOGFILE
```

**Step 2c:**

```bash
OUTPUT=HSC/runs/dw1008p7058 \
LOGFILE=$LOGDIR/dw1008p7058_step2c.log; \
date | tee $LOGFILE; \
pipetask --long-log run --register-dataset-types -j 24 \
-b $REPO --instrument lsst.obs.subaru.HyperSuprimeCam \
-i HSC/raw/all,HSC/calib/dw1008p7058,HSC/calib,HSC/calib/unbounded,refcats/gen2,skymaps \
-o $OUTPUT \
-p $REPO/pipelines/step2ce.yaml#step2c \
-c updateVisitSummary:wcs_provider="input_summary" \
-c updateVisitSummary:photo_calib_provider="input_summary" \
-c updateVisitSummary:background_provider="input_summary" \
-d "instrument='HSC' AND skymap='hsc_rings_v1' AND visit IN $SCIEXPS" \
2>&1 | tee -a $LOGFILE; \
date | tee -a $LOGFILE
```

**Step 2e:**
```bash
OUTPUT=HSC/runs/dw1008p7058 \
LOGFILE=$LOGDIR/dw1008p7058_step2e.log; \
date | tee $LOGFILE; \
pipetask --long-log run --register-dataset-types -j 24 \
-b $REPO --instrument lsst.obs.subaru.HyperSuprimeCam \
-i HSC/raw/all,HSC/calib/dw1008p7058,HSC/calib,HSC/calib/unbounded,refcats/gen2,skymaps \
-o $OUTPUT \
-p $REPO/pipelines/step2ce.yaml#step2e \
-d "instrument='HSC' AND skymap='hsc_rings_v1'" \
2>&1 | tee -a $LOGFILE; \
date | tee -a $LOGFILE
```


**Step 3a:**

```bash
pipetask build -p $REPO/pipelines/step3a.yaml --show config | less

pipetask build \
-p $REPO/pipelines/step3a.yaml#step3a \
--show pipeline \
--pipeline-dot ./pipeline.dot; \
dot ./pipeline.dot -Tpdf > $LOGDIR/pipeline_step3a.pdf
cp $LOGDIR/pipeline_step3a.pdf /tigress/jiaxuanl/public_html/LSST/HSC/NGC4605/

```
<!-- , 17115, 17114 -->

```bash
TRACTS="(18360, 18359, 18358, 18440, 18441, 18439, 18442)" \
OUTPUT=HSC/runs/dw1008p7058 \
LOGFILE=$LOGDIR/dw1008p7058_step3a.log; \
date | tee $LOGFILE; \
pipetask --long-log run --register-dataset-types -j 24 \
-b $REPO --instrument lsst.obs.subaru.HyperSuprimeCam \
-i HSC/raw/all,HSC/calib/dw1008p7058,HSC/calib,HSC/calib/unbounded,refcats/gen2,skymaps \
-o $OUTPUT \
-p $REPO/pipelines/step3a.yaml#step3a \
-d "instrument='HSC' AND skymap='hsc_rings_v1' AND tract IN $TRACTS" \
--clobber-outputs --extend-run --skip-existing \
2>&1 | tee -a $LOGFILE; \
date | tee -a $LOGFILE
```

In [ ]:
butler query-dimension-records $REPO exposure \
--where "instrument='HSC' AND exposure.observation_type='zero' AND band='z'"

In [ ]:
SCIEXPS="(284652, 285228, 285232, 308696)" \
OUTPUT=HSC/runs/dw1336p5226 \
LOGFILE=$LOGDIR/dw1336p5226_step1.log; \
date | tee $LOGFILE; \
pipetask --long-log run --register-dataset-types -j 48 \
-b $REPO --instrument lsst.obs.subaru.HyperSuprimeCam \
-i HSC/raw/all,HSC/calib/dw1336p5226,HSC/calib,HSC/calib/unbounded,refcats/gen2,skymaps \
-o $OUTPUT \
-p $DRP_PIPE_DIR/pipelines/HSC/DRP-RC2.yaml#step1 \
-d "instrument='HSC' AND exposure in $SCIEXPS AND detector!=9 AND detector!=33" \
--clobber-outputs --extend-run --skip-existing \
2>&1 | tee -a $LOGFILE; \
date | tee -a $LOGFILE

In [173]:
# fits.open('/scratch/gpfs/JENNYG/jiaxuanl/HSC/raw_smoka/dw1008p7058/HSCA30780700.fits')[0].header['DATA-TYP']

# dw0108p1650 and dw0106p1616b

dw0108p1650: `17.231417225822774, 16.838790960356715`
dw0106p1616b: `16.5218643374393, 16.28085595829385`
They are in the same frame.

 - science frames: 297998 (in g-band)
 - domeflats: 299084..299103
 - bias: 297040..297059 (these are R2)
 - dark: 299048..299057
 - `P03jiaxuanl1228052246NT`


the darks are `281302, 281304, 281306, 281308, 281310`, which are already reduced in `dw1008p7058`. So how to use these darks for this object will be a (small) problem. 

In [158]:
# dw0108p1650
expid_dict = {'bias': [(307778, 307796+1)],
              'flat': [(276834, 276852+1), (299084, 299102+1)], 
              'dark': [(281302, 281310+1)],
              'science': [(297998, 297999)]}

print(ingest_files("dw0108p1650", expid_dict['bias'], filetype='bias'))
print(ingest_files("dw0108p1650", expid_dict['flat'], filetype='flat'))
print(ingest_files("dw0108p1650", expid_dict['dark'], filetype='dark'))
print(ingest_files("dw0108p1650", expid_dict['science'], filetype='science'))

######## Ingesting bias... ##########
mkdir -p /scratch/gpfs/JENNYG/jiaxuanl/HSC/raw_smoka/dw0108p1650/bias
for expid in {307778..307797}; do
    mv /scratch/gpfs/JENNYG/jiaxuanl/HSC/raw_smoka/dw0108p1650/HSCA${expid}*.fits \
       /scratch/gpfs/JENNYG/jiaxuanl/HSC/raw_smoka/dw0108p1650/bias/
done

LOGFILE=$LOGDIR/dw0108p1650_ingest_bias.log; \
FILES=/scratch/gpfs/JENNYG/jiaxuanl/HSC/raw_smoka/dw0108p1650/bias/*.fits; \
date | tee $LOGFILE; \
butler ingest-raws $REPO $FILES --transfer link \
2>&1 | tee -a $LOGFILE; \
date | tee -a $LOGFILE

######## Ingesting flat... ##########
mkdir -p /scratch/gpfs/JENNYG/jiaxuanl/HSC/raw_smoka/dw0108p1650/flat
for expid in {276834..276853} {299084..299103}; do
    mv /scratch/gpfs/JENNYG/jiaxuanl/HSC/raw_smoka/dw0108p1650/HSCA${expid}*.fits \
       /scratch/gpfs/JENNYG/jiaxuanl/HSC/raw_smoka/dw0108p1650/flat/
done

LOGFILE=$LOGDIR/dw0108p1650_ingest_flat.log; \
FILES=/scratch/gpfs/JENNYG/jiaxuanl/HSC/raw_smoka/dw0108p1650/flat/*.fits; \
date | tee

In [ ]:
LOGFILE=$LOGDIR/NGC4605_define_visits.log; \
date | tee $LOGFILE; \
butler define-visits $REPO lsst.obs.subaru.HyperSuprimeCam \
2>&1 | tee -a $LOGFILE; \
date | tee -a $LOGFILE

In [ ]:
LOGFILE=$LOGDIR/dw0108p1650_ingest_science.log; \
FILES=/scratch/gpfs/JENNYG/jiaxuanl/HSC/raw_smoka/dw0108p1650/science/*.fits; \
date | tee $LOGFILE; \
butler ingest-raws $REPO $FILES --transfer link \
2>&1 | tee -a $LOGFILE; \
date | tee -a $LOGFILE

In [ ]:
butler query-dimension-records $REPO exposure \
--where "instrument='HSC' AND exposure.observation_type='science' AND band='g'"


In [103]:
galaxy_name = "dw0108p1650"

command = calibration_command(galaxy_name, expid_dict, filetype="bias", clobber=True)
print(command)

######## Generating bias calibration command... ##########
export BIASEXPS="(307778, 307780, 307782, 307784, 307786, 307788, 307790, 307792, 307794, 307796)"
LOGFILE=$LOGDIR/dw0108p1650_bias.log; \
date | tee $LOGFILE; \
pipetask --long-log run --register-dataset-types -j 36 \
-b $REPO --instrument lsst.obs.subaru.HyperSuprimeCam \
-i HSC/raw/all,HSC/calib,HSC/calib/unbounded,HSC/calib/curated/19700101T000000Z \
-o HSC/calib/dw0108p1650/bias \
-p $CP_PIPE_DIR/pipelines/HSC/cpBias.yaml \
-d "instrument='HSC' AND exposure IN $BIASEXPS AND detector!=9" \
--clobber-outputs --extend-run --skip-existing \
-c cpBiasCombine:checkNoData=False \
2>&1 | tee -a $LOGFILE; \
date | tee -a $LOGFILE\

butler certify-calibrations $REPO HSC/calib/dw0108p1650/bias HSC/calib/dw0108p1650 bias --begin-date 2016-01-01T00:00:00 --end-date 2024-06-30T23:59:59



In [107]:
command = calibration_command(galaxy_name, expid_dict, filetype="sky", clobber=False)
print(command)

######## Generating sky calibration command... ##########
export SKYEXPS="(297998)"
LOGFILE=$LOGDIR/dw0108p1650_sky.log; \
date | tee $LOGFILE; \
pipetask --long-log run --register-dataset-types -j 36 \
-b $REPO --instrument lsst.obs.subaru.HyperSuprimeCam \
-i HSC/raw/all,HSC/calib,HSC/calib/unbounded,HSC/calib/dw0108p1650,HSC/calib/curated/19700101T000000Z \
-o HSC/calib/dw0108p1650/sky \
-p $CP_PIPE_DIR/pipelines/HSC/cpSky.yaml \
-d "instrument='HSC' AND exposure IN $SKYEXPS AND detector!=9" \
2>&1 | tee -a $LOGFILE; \
date | tee -a $LOGFILE\

butler certify-calibrations $REPO HSC/calib/dw0108p1650/sky HSC/calib/dw0108p1650 sky --begin-date 2016-01-01T00:00:00 --end-date 2024-06-30T23:59:59



**Get a computing node:**
`salloc --nodes=1 --ntasks=1 --mem=32G --time=08:00:00 --account=merian`


**Step 1**

```bash
SCIEXPS="(297998)" \
OUTPUT=HSC/runs/dw0108p1650 \
LOGFILE=$LOGDIR/dw0108p1650_step1.log; \
date | tee $LOGFILE; \
pipetask --long-log run --register-dataset-types -j 48 \
-b $REPO --instrument lsst.obs.subaru.HyperSuprimeCam \
-i HSC/raw/all,HSC/calib/dw0108p1650,HSC/calib,HSC/calib/unbounded,refcats/gen2,skymaps \
-o $OUTPUT \
-p $DRP_PIPE_DIR/pipelines/HSC/DRP-RC2.yaml#step1 \
-d "instrument='HSC' AND exposure IN $SCIEXPS AND detector!=9" \
2>&1 | tee -a $LOGFILE; \
date | tee -a $LOGFILE
```

**Step 2a**
```bash
BADCCDS="(9, 33, 104, 105, 106, 107, 108, 109, 110, 111)" \
SCIEXPS="(297998)" \
OUTPUT=HSC/runs/dw0108p1650 \
LOGFILE=$LOGDIR/dw0108p1650_step2a.log; \
date | tee $LOGFILE; \
pipetask --long-log run --register-dataset-types -j 48 \
-b $REPO --instrument lsst.obs.subaru.HyperSuprimeCam \
-i HSC/raw/all,HSC/calib/dw0108p1650,HSC/calib,HSC/calib/unbounded,refcats/gen2,skymaps \
-o $OUTPUT \
-p $DRP_PIPE_DIR/pipelines/HSC/DRP-RC2.yaml#step2a \
-d "instrument='HSC' AND exposure IN $SCIEXPS AND detector NOT IN $BADCCDS" \
2>&1 | tee -a $LOGFILE; \
date | tee -a $LOGFILE
```

**Step 2b**

```
pipetask build -p $REPO/pipelines/step2b.yaml#step2b --show config | less
```

```bash
OUTPUT=HSC/runs/dw0108p1650 \
LOGFILE=$LOGDIR/dw0108p1650_step2b.log; \
date | tee $LOGFILE; \
pipetask --long-log run --register-dataset-types -j 48 \
-b $REPO --instrument lsst.obs.subaru.HyperSuprimeCam \
-i HSC/raw/all,HSC/calib/dw0108p1650,HSC/calib,HSC/calib/unbounded,refcats/gen2,skymaps \
-o $OUTPUT \
-p $REPO/pipelines/step2b.yaml#step2b \
-d "instrument='HSC' AND skymap='hsc_rings_v1'" \
-c jointcal:connections.astrometryRefCat='gaia_dr3_20230707' \
-c jointcal:maxAstrometrySteps=50 \
-c jointcal:doPhotometry=True \
2>&1 | tee -a $LOGFILE; \
date | tee -a $LOGFILE
```

**Step 2c:**

```bash
OUTPUT=HSC/runs/dw0108p1650 \
LOGFILE=$LOGDIR/dw0108p1650_step2c.log; \
date | tee $LOGFILE; \
pipetask --long-log run --register-dataset-types -j 24 \
-b $REPO --instrument lsst.obs.subaru.HyperSuprimeCam \
-i HSC/raw/all,HSC/calib/dw0108p1650,HSC/calib,HSC/calib/unbounded,refcats/gen2,skymaps \
-o $OUTPUT \
-p $REPO/pipelines/step2ce.yaml#step2c \
-c updateVisitSummary:wcs_provider="input_summary" \
-c updateVisitSummary:photo_calib_provider="input_summary" \
-c updateVisitSummary:background_provider="input_summary" \
-d "instrument='HSC' AND skymap='hsc_rings_v1' AND visit IN $SCIEXPS" \
2>&1 | tee -a $LOGFILE; \
date | tee -a $LOGFILE
```

**Step 2e:**
```bash
OUTPUT=HSC/runs/dw0108p1650 \
LOGFILE=$LOGDIR/dw0108p1650_step2e.log; \
date | tee $LOGFILE; \
pipetask --long-log run --register-dataset-types -j 24 \
-b $REPO --instrument lsst.obs.subaru.HyperSuprimeCam \
-i HSC/raw/all,HSC/calib/dw0108p1650,HSC/calib,HSC/calib/unbounded,refcats/gen2,skymaps \
-o $OUTPUT \
-p $REPO/pipelines/step2ce.yaml#step2e \
-d "instrument='HSC' AND skymap='hsc_rings_v1'" \
2>&1 | tee -a $LOGFILE; \
date | tee -a $LOGFILE
```


**Step 3a:**

```bash
pipetask build -p $REPO/pipelines/step3a.yaml --show config | less

pipetask build \
-p $REPO/pipelines/step3a.yaml#step3a \
--show pipeline \
--pipeline-dot ./pipeline.dot; \
dot ./pipeline.dot -Tpdf > $LOGDIR/pipeline_step3a.pdf
cp $LOGDIR/pipeline_step3a.pdf /tigress/jiaxuanl/public_html/LSST/HSC/NGC4605/

```
<!-- , 17115, 17114 -->

```bash
TRACTS="(11960, 11959, 12193, 12194)" \
OUTPUT=HSC/runs/dw0108p1650 \
LOGFILE=$LOGDIR/dw0108p1650_step3a.log; \
date | tee $LOGFILE; \
pipetask --long-log run --register-dataset-types -j 24 \
-b $REPO --instrument lsst.obs.subaru.HyperSuprimeCam \
-i HSC/raw/all,HSC/calib/dw0108p1650,HSC/calib,HSC/calib/unbounded,refcats/gen2,skymaps \
-o $OUTPUT \
-p $REPO/pipelines/step3a.yaml#step3a \
-d "instrument='HSC' AND skymap='hsc_rings_v1' AND tract IN $TRACTS" \
--clobber-outputs --extend-run --skip-existing \
2>&1 | tee -a $LOGFILE; \
date | tee -a $LOGFILE
```